In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

In [15]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",                 # ✅ required
            "command": "uvx",                     # uvx will run the CLI from PyPI
            "args": ["mcp-server-time", "--local-timezone=Asia/Dhaka"],
            # Optional:
            # "env": {"PYTHONUNBUFFERED": "1"},
            # "request_timeout": 120,
            # "idle_timeout": 300,
        },
        "edgeone-pages-mcp-server": {
            "transport": "stdio",
            "command": "npx",
            "args": ["edgeone-pages-mcp"],
            # If your npx command needs env, you can add:
            # "env": {"NODE_OPTIONS": "..."}
        },
    }
)

In [16]:
tools = await client.get_tools()

In [17]:
tools

[StructuredTool(name='get_current_time', description='Get current time in a specific timezones', args_schema={'type': 'object', 'properties': {'timezone': {'type': 'string', 'description': "IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'Asia/Dhaka' as local timezone if no timezone provided by the user."}}, 'required': ['timezone']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x75a9f3f0c400>),
 StructuredTool(name='convert_time', description='Convert time between timezones', args_schema={'type': 'object', 'properties': {'source_timezone': {'type': 'string', 'description': "Source IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'Asia/Dhaka' as local timezone if no source timezone provided by the user."}, 'time': {'type': 'string', 'description': 'Time to convert in 24-hour format (HH:MM)'}, 'target_timezone': {'type': 'string', 'description': "Target IANA timezone name (e.g

In [14]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent("gpt-4o-mini", tools)
response = await agent.ainvoke({'messages': [{'role': 'user', 'content': "Whats the current time in Dhaka?"}]})
response['messages'][-1].content

'The current time in Dhaka is 09:44 AM on September 6, 2025.'

In [18]:
def _gradio_history_to_messages(history):
    """Convert Gradio history (tuples or dicts) into LangGraph MessagesState list."""
    messages = []
    for turn in history:
        # Tuple/list style: (user, assistant)
        if isinstance(turn, (list, tuple)) and len(turn) == 2:
            user_msg, assistant_msg = turn
            if user_msg:
                messages.append({"role": "user", "content": user_msg})
            if assistant_msg:
                messages.append({"role": "assistant", "content": assistant_msg})
        # Dict style: {"role": "...", "content": "..."} (some Gradio configs/plugins)
        elif isinstance(turn, dict) and "role" in turn and "content" in turn:
            messages.append({"role": turn["role"], "content": turn["content"]})
    return messages

In [19]:
import gradio as gr

async def chat(user_message, history):
    messages = _gradio_history_to_messages(history)
    messages.append({"role": "user", "content": user_message})

    # Call your LangGraph agent
    result = await agent.ainvoke({"messages": messages})

    # Return only the assistant text; ChatInterface updates history itself
    return result["messages"][-1].content


gr.ChatInterface(chat, title="LangGraph MCP Agent").launch()

/mnt/d/Work/Projects/Ubuntu_Home/projects/ownership-ai/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
